# Module 2 – Session 4: Practical Exercises

## Exercise 1 — Trace A* Search (Conceptual)


#### Given Graph

```
S (h=10)
|-- (cost 4) -- A (h=6)
| |-- (cost 8) -- C (h=3)
| | |-- (cost 3) -- G (h=0)
| |-- (cost 2) -- D (h=4)
| |-- (cost 10) -- G (h=0)
|-- (cost 5) -- B (h=7)
|-- (cost 6) -- E (h=1)
|-- (cost 2) -- G (h=0)
```

Start node → **S**  
Goal node → **G**

**Step 1 — Start from S**  
- g(S)=0, h(S)=10 ⇒ f(S)=10  
Expand neighbors of S:

| Neighbor | Edge cost | g(n) | h(n) | f(n)=g+h |
|---------:|----------:|-----:|-----:|---------:|
| A        | 4         | 4    | 6    | 10       |
| B        | 5         | 5    | 7    | 12       |
| E        | 6         | 6    | 1    | 7        |
| G        | 2         | 2    | 0    | 2        |


Open (by f): `[ G(2), E(7), A(10), B(12) ]`  
Closed: `[ S ]`

**Step 2 — Pop the smallest f**  
- Choose **G (f=2)**. It is the goal → **stop**.

### Result
- Expanded nodes: **S**, **G**  
- Final path: **S → G**  
- Total cost: **2**


### Comparison with Greedy Best-First Search

| Algorithm | Selection criterion | Found path | Cost | Comment |
|------------|--------------------|-------------|-------|----------|
| Greedy Best-First | Chooses smallest h(n) | S → G | 2 | Greedy ignores g(n); not always optimal |
| A* Search | Chooses smallest f(n)=g+h | S → G | 2 | Combines actual and heuristic costs → optimal |


##### Discussion

In this graph, both Greedy and A* found the same path because there is a **direct edge S→G** with the lowest possible cost (2).  
However, if that direct link didn’t exist, Greedy would probably choose a node with the smallest h(n), while A* would keep exploring based on both g and h, finding the **true shortest path**.

**Key Takeaway:**  
> A* search is *complete* and *optimal* when h(n) is admissible (i.e., never overestimates the real cost).

---

##### Summary
- Expanded nodes → S, G  
- Final path → S → G  
- Total cost → 2  
- A* found optimal path (same as Greedy here)

### Exercise 2 — Implement A* Search (Coding)


#####  Step 1 — Define the graph and heuristic values

Graph uses **real road costs** (g).  
Heuristics are straight-line estimates to **Dilizhan** (h).

In [17]:
# Actual road costs between Armenian cities
graph = {
    'Yerevan': {'Gyumri': 20, 'Sevan': 60},
    'Gyumri': {'Yerevan': 20, 'Vanadzor': 40},
    'Sevan': {'Yerevan': 60, 'Dilizhan': 40},
    'Vanadzor': {'Gyumri': 40, 'Dilizhan': 50},
    'Dilizhan': {'Sevan': 40, 'Vanadzor': 50}
}

# Heuristic values (straight-line distance to Dilizhan)
heuristics = {
    'Yerevan': 90,
    'Gyumri': 85,
    'Sevan': 25,
    'Vanadzor': 35,
    'Dilizhan': 0
}

start = 'Yerevan'
goal = 'Dilizhan'


##### Step 2 — Implement the A* search function

We maintain an Open List (Priority Queue) by **smallest f(n)=g+h**.  
We also track `visited` to avoid re-expanding nodes.

In [18]:
from queue import PriorityQueue

def a_star_search(graph, start, goal, heuristics):
    open_list = PriorityQueue()
    open_list.put((heuristics[start], [start]))   # initial f = 0 + h(start)
    visited = set()

    while not open_list.empty():
        f_value, path = open_list.get()
        node = path[-1]

        if node == goal:
            return path  # we will compute g separately

        if node in visited:
            continue
        visited.add(node)

        # current g cost of the path so far
        g_cost = sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))

        for nb, cost in graph[node].items():
            if nb not in visited:
                g_new = g_cost + cost
                f_new = g_new + heuristics[nb]
                open_list.put((f_new, path + [nb]))

    return None

def path_cost(graph, path):
    if not path or len(path) < 2:
        return 0
    return sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))


##### Step 3 — Run A* and print the result


In [19]:
path_a = a_star_search(graph, start, goal, heuristics)
cost_a = path_cost(graph, path_a)

print("A* path:", " → ".join(path_a))
print("A* total cost (g):", cost_a)

A* path: Yerevan → Sevan → Dilizhan
A* total cost (g): 100


##### 🔁 Step 4 — Compare with BFS (unit costs) and Greedy Best-First

- **BFS** assumes all edges have the same cost (unit weights) → shortest by number of edges  
- **Greedy** always picks the smallest **h(n)** next  
We’ll run both on the same graph to answer requirement (3).

In [20]:
from collections import deque
from queue import PriorityQueue

# Build unweighted adjacency for BFS (ignore edge weights)
adj = {u: list(graph[u].keys()) for u in graph}

def bfs_unit_cost(adj, start, goal):
    q = deque([[start]])
    visited = set([start])
    expanded = 0
    while q:
        path = q.popleft()
        node = path[-1]
        expanded += 1
        if node == goal:
            return path, expanded
        for nb in adj[node]:
            if nb not in visited:
                visited.add(nb)
                q.append(path + [nb])
    return None, expanded

def greedy_best_first(graph, start, goal, heuristics):
    pq = PriorityQueue()
    pq.put((heuristics[start], [start]))
    visited = set()
    expanded = 0
    while not pq.empty():
        h, path = pq.get()
        node = path[-1]
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        if node == goal:
            return path, expanded
        for nb in graph[node].keys():
            if nb not in visited:
                pq.put((heuristics[nb], path + [nb]))
    return None, expanded

bfs_path, bfs_exp = bfs_unit_cost(adj, start, goal)
greedy_path, greedy_exp = greedy_best_first(graph, start, goal, heuristics)

print("BFS (unit costs) path:", " → ".join(bfs_path), "| expanded:", bfs_exp)
print("Greedy path:", " → ".join(greedy_path), "| expanded:", greedy_exp)


BFS (unit costs) path: Yerevan → Sevan → Dilizhan | expanded: 5
Greedy path: Yerevan → Sevan → Dilizhan | expanded: 3


##### Step 5 — Dijkstra & efficiency measurement (requirement 5)

We will compare **A\*** and **Dijkstra** by:
- **expanded nodes** count
- optional **runtime** (ms)

In [21]:
import time
from queue import PriorityQueue

def a_star_with_metrics(graph, start, goal, heuristics):
    open_list = PriorityQueue()
    open_list.put((heuristics[start], [start]))
    visited = set()
    expanded = 0
    t0 = time.perf_counter()
    while not open_list.empty():
        f_value, path = open_list.get()
        node = path[-1]
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        if node == goal:
            g = path_cost(graph, path)
            t1 = time.perf_counter()
            return path, g, expanded, (t1 - t0)*1000
        g_so_far = path_cost(graph, path)
        for nb, cost in graph[node].items():
            if nb not in visited:
                g_new = g_so_far + cost
                f_new = g_new + heuristics[nb]
                open_list.put((f_new, path + [nb]))
    t1 = time.perf_counter()
    return None, float('inf'), expanded, (t1 - t0)*1000

def dijkstra_with_metrics(graph, start, goal):
    pq = PriorityQueue()
    pq.put((0, [start]))  # (g, path)
    visited = set()
    expanded = 0
    t0 = time.perf_counter()
    while not pq.empty():
        g, path = pq.get()
        node = path[-1]
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        if node == goal:
            t1 = time.perf_counter()
            return path, g, expanded, (t1 - t0)*1000
        for nb, cost in graph[node].items():
            if nb not in visited:
                pq.put((g + cost, path + [nb]))
    t1 = time.perf_counter()
    return None, float('inf'), expanded, (t1 - t0)*1000

ap, ag, ae, ams = a_star_with_metrics(graph, start, goal, heuristics)
dp, dg, de, dms = dijkstra_with_metrics(graph, start, goal)

print("A*       :", " → ".join(ap), "| g =", ag, "| expanded =", ae, "| ~%.3f ms" % ams)
print("Dijkstra :", " → ".join(dp), "| g =", dg, "| expanded =", de, "| ~%.3f ms" % dms)


A*       : Yerevan → Sevan → Dilizhan | g = 100 | expanded = 3 | ~0.026 ms
Dijkstra : Yerevan → Sevan → Dilizhan | g = 100 | expanded = 5 | ~0.013 ms


##### Step 6 — Modify costs/heuristics so A* picks another optimal path (requirement 4)

Two minimal options:
1) **Heuristic tweak only:** lower `h(Gyumri)` and raise `h(Sevan)` → biases A* towards `Yerevan → Gyumri → Vanadzor → Dilizhan`.
2) **Small cost tweak:** increase `Sevan → Dilizhan` from `40` to `55` to make the Gyumri/Vanadzor route competitive.


In [22]:
# (1) Heuristic-only tweak
h_tweak = dict(heuristics)
h_tweak['Gyumri'] = 20   # much closer
h_tweak['Sevan']  = 60   # farther

p1 = a_star_search(graph, start, goal, h_tweak)
print("Heuristic tweak → A* path:", " → ".join(p1), "| g =", path_cost(graph, p1))

# (2) Small cost tweak
g_tweak = {u: dict(v) for u, v in graph.items()}
g_tweak['Sevan']['Dilizhan'] = 55  # 40 → 55

p2 = a_star_search(g_tweak, start, goal, heuristics)
print("Cost tweak → A* path:", " → ".join(p2), "| g =", path_cost(g_tweak, p2))


Heuristic tweak → A* path: Yerevan → Gyumri → Vanadzor → Dilizhan | g = 110
Cost tweak → A* path: Yerevan → Gyumri → Vanadzor → Dilizhan | g = 110


## Summary

- **Exercise 1 (Trace):** Expanded S, G → Path `S → G` → Cost `2`
- **Exercise 2 (Code):** A* on Armenian cities → `Yerevan → Sevan → Dilizhan` → Cost `100`
- **BFS (unit costs):** same path by 2 edges; **Greedy** also picks the same due to smallest h
- **Dijkstra vs A*:** both optimal g, but A* tends to expand fewer nodes (uses h)
- **Tweaks:** small heuristic/cost changes can steer A* to a different (still optimal or suboptimal) route depending on admissibility of h

**Key idea:** A* = Dijkstra + admissible heuristic → optimal & efficient.
